In [1]:
# create circle 
import numpy as np
import gmsh

def create_surface(lines, arcs_1, arcs_2, radials,i,n_circle):

    i = i % n_circle
    j = (i-1) % n_circle

    loop = gmsh.model.occ.addCurveLoop([lines[j], 
                                        arcs_2[j],
                                        lines[i],
                                        arcs_1[j]
                                        ])
    gmsh.model.occ.synchronize()
    face_1 = gmsh.model.occ.addPlaneSurface([loop])
    gmsh.model.occ.synchronize()

    loop = gmsh.model.occ.addCurveLoop([radials[j],
                                        radials[i],
                                        arcs_2[j],
                                        ])
    gmsh.model.occ.synchronize()
    face_2 = gmsh.model.occ.addPlaneSurface([loop])
    gmsh.model.occ.synchronize()

    return face_1, face_2

def circle(r1,r2,x0,y0,z0,n_circle):

    if r1 < r2:
        print("r1 should be larger than r2")
        return
    
    angle_span = np.linspace(0, 2*np.pi, n_circle+1)
    angle_span = angle_span[:-1]
    x1 = r1*np.cos(angle_span) 
    y1 = r1*np.sin(angle_span) 
    z1 = 0*angle_span
    
    x2 = r2*np.cos(angle_span) 
    y2 = r2*np.sin(angle_span) 
    z2 = 0*angle_span

    # vec_norm = np.array([0,1,0])
    # vec_norm = vec_norm/np.linalg.norm(vec_norm)
    # # rotate and translate
    # mat_rot = np.array([[0, -vec_norm[2], vec_norm[1]],
    #                     [vec_norm[2], 0, -vec_norm[0]],
    #                     [-vec_norm[1], vec_norm[0], 0]])
    
    # x1,y1,z1 = np.dot(mat_rot, np.array([x1,y1,z1]))
    # x2,y2,z2 = np.dot(mat_rot, np.array([x2,y2,z2]))

    x1 = x1 + x0
    y1 = y1 + y0
    z1 = z1 + z0

    x2 = x2 + x0
    y2 = y2 + y0
    z2 = z2 + z0


    #center 
    tg_center = gmsh.model.occ.addPoint(x0, y0, z0)

    tags_1 = []
    tags_2 = []
    for i in range(n_circle):
        tg_1 = gmsh.model.occ.addPoint(x1[i], y1[i], z1[i])
        tg_2 = gmsh.model.occ.addPoint(x2[i], y2[i], z2[i])
        tags_1.append(tg_1)
        tags_2.append(tg_2)
    # create arc 
    # sync
    gmsh.model.occ.synchronize()
    # gmsh.fltk.run()
    # create circle from arc 
    arcs_1  = []
    arcs_2  = []
    lines   = []
    radials = []

    wires_arc_1  = []
    wires_arc_2  = []
    wires_line   = []
    wires_radial = []

    surfaces = []

    for i in range(n_circle):

        arc_1 = gmsh.model.occ.addCircleArc(tags_1[i], 
                                            tg_center, 
                                            tags_1[(i+1)%n_circle])
        
        arc_2 = gmsh.model.occ.addCircleArc(tags_2[i], 
                                            tg_center, 
                                            tags_2[(i+1)%n_circle])
        arcs_1.append(arc_1)
        arcs_2.append(arc_2)

        # line 
        line = gmsh.model.occ.addLine(tags_1[i], tags_2[i])
        lines.append(line)

        radial = gmsh.model.occ.addLine(tg_center,tags_2[i])
        radials.append(radial)

        gmsh.model.occ.synchronize()
        # gmsh.fltk.run()

        wire_arc_1  = gmsh.model.occ.addWire([arc_1])
        wire_arc_2  = gmsh.model.occ.addWire([arc_2])
        wire_line   = gmsh.model.occ.addWire([line])
        wire_radial = gmsh.model.occ.addWire([radial])
    
        gmsh.model.occ.synchronize()

        wires_arc_1.append(wire_arc_1)
        wires_arc_2.append(wire_arc_2)
        wires_line.append(wire_line)
        wires_radial.append(wire_radial)

        if i > 0:
            # face 

            f1,f2 = create_surface(lines, 
                           arcs_1, arcs_2, 
                           radials,
                           i,n_circle)
            surfaces.append(f1)
            surfaces.append(f2)

    f1,f2 = create_surface(lines, 
                   arcs_1, arcs_2, 
                   radials,
                   i+1,n_circle)
    surfaces.append(f1)
    surfaces.append(f2)

    wires_all = wires_arc_1 + wires_arc_2 + wires_line + wires_radial
    return {
        "arcs_1": arcs_1,
        "arcs_2": arcs_2,
        "lines": lines,
        "radials": radials,
        "surfaces": surfaces,
        "wires": wires_all,
        "points":{
            "center": tg_center,
            "external": tags_1,
            "internal": tags_2
        }
    }

In [2]:
gmsh.initialize()
gmsh.clear()

# set option visualizaton 
# show surface

results_1 = circle(r1=1,r2=0.8, x0=0, y0=0,z0=3, n_circle=10)
results_2 = circle(r1=1,r2=0.8, x0=0, y0=0,z0=0, n_circle=10)

In [3]:
points_1 = results_1["points"]["external"]
points_2 = results_2["points"]["external"]
line_external = []
for i,j in zip(points_1, points_2):
    le = gmsh.model.occ.addLine(i,j)
    gmsh.model.occ.addWire([le])
    line_external.append(le)

points_1 = results_1["points"]["internal"]
points_2 = results_2["points"]["internal"]
line_internal = []
for i,j in zip(points_1, points_2):
    li = gmsh.model.occ.addLine(i,j)
    gmsh.model.occ.addWire([li])
    line_internal.append(li)

# sync

nucleo_line = gmsh.model.occ.addLine(results_1["points"]["center"], 
                                results_2["points"]["center"])

wire_nucleo = gmsh.model.occ.addWire([nucleo_line])


gmsh.model.occ.synchronize()

for rd1,rd2,li in zip(results_1["radials"],results_2["radials"],line_internal):

    lines = [nucleo_line,rd1,li,rd2]
    # face 
    loop = gmsh.model.occ.addCurveLoop(lines)
    gmsh.model.occ.synchronize()
    face = gmsh.model.occ.addPlaneSurface([loop])
    gmsh.model.occ.synchronize()

for r1_li,le,li,r2_li in zip(results_1["lines"],
                             line_external,
                             line_internal,
                             results_2["lines"]):

    lines = [r1_li,
             le,
             r2_li,
             li]
    # face 
    loop = gmsh.model.occ.addCurveLoop(lines)
    gmsh.model.occ.synchronize()
    face = gmsh.model.occ.addPlaneSurface([loop])
    gmsh.model.occ.synchronize()



gmsh.model.occ.synchronize()
# mesh 
gmsh.model.mesh.generate(2) 
# recombine
# gmsh.model.mesh.recombine()
gmsh.model.occ.synchronize()


In [4]:
lines

[39, 90, 79, 100]

In [5]:
gmsh.model.occ.synchronize()


In [8]:
gmsh.fltk.run()